# Logistic Regression (LogReg) Example

This is a SIMPLE sample implementation of Logistic Regression using the `turtles` `LogReg` class and various supporting functions.

The `LogReg` class implements Logistic Regression using Maximum Likelihood Estimation (MLE) for parameter estimation.

In [1]:
import time

import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

from turtles.stats.glms import LogReg
from turtles.stats import (
    variance_inflation_factor,
    pca
)

In [2]:
random_state = 5
test_size = 0.2

In [3]:
# load sample data
X, y = load_breast_cancer(return_X_y=True)

print("X:", X.shape)
print("y:", y.shape)

X: (569, 30)
y: (569,)


In [4]:
# create splits
Xtrain, Xtest, Ytrain, Ytest = train_test_split(
    X, 
    y, 
    test_size=test_size, 
    random_state=random_state
)

Ytrain = Ytrain.reshape(Ytrain.shape[0], 1)
Ytest = Ytest.reshape(Ytest.shape[0], 1)

In [5]:
# EDA

# check for multicollinearity
vifs = variance_inflation_factor(Xtrain)
print("VIF Table:")
display(vifs)

VIF Table:


,Coefficient,R-squared,VIF
0,x0,0.999729,3692.842120
1,x1,0.911654,11.319176
2,x2,0.999730,3702.834171
3,x3,0.996914,324.095944
4,x4,0.878489,8.229712
5,x5,0.977671,44.784059
6,x6,0.986572,74.472502
7,x7,0.982589,57.434044
8,x8,0.763329,4.225272
9,x9,0.932192,14.747468


In [6]:
# there appears to be significant multicollinearity
# let's use PCA to reduce the dimensions, then re-split the data
X = pca(X, 2)

Xtrain, Xtest, Ytrain, Ytest = train_test_split(
    X, 
    y, 
    test_size=0.2, 
    random_state=5
)

Ytrain = Ytrain.reshape(Ytrain.shape[0], 1)
Ytest = Ytest.reshape(Ytest.shape[0], 1)

print("Xtrain:", Xtrain.shape)
print("Xtest:", Xtest.shape)
print("Ytrain:", Ytrain.shape)
print("Ytest", Ytest.shape)
print()

Xtrain: (455, 2)
Xtest: (114, 2)
Ytrain: (455, 1)
Ytest (114, 1)



In [7]:
# fit model using gradient descent
# tune hyperparameters as needed
g_model = LogReg(
    method="grad", 
    learning_rate=0.1,
    tolerance=0.000001
)

In [8]:
start = time.time()
g_model.fit(Xtrain, Ytrain, var_names=["PC1", "PC2"])
print("Gradient Descent time to fit:", time.time() - start)
print("Gradient Descent Iterations:", g_model.iterations)
print()
print("Gradient Descent Model Summary:\n")
display(g_model.summary())

Gradient Descent time to fit: 0.04040074348449707
Gradient Descent Iterations: 272

Gradient Descent Model Summary:



,Variable,Coefficient,Std Error,z-statistic,p-value,[0.025,0.075]
0,Intercept,-0.8608,0.2993,-2.8761,0.004,-1.4475,-0.2742
1,PC1,7.9024,0.8969,8.8111,0.000,6.1446,9.6602
2,PC2,2.4758,0.4376,5.6576,0.000,1.6181,3.3335


In [9]:
# fit model using newtons method
# tune hyperparameters as needed
n_model = LogReg(
    method="newton",
    learning_rate=0.1,
    tolerance=0.00001
)

In [10]:
start = time.time()
n_model.fit(Xtrain, Ytrain)
print("Newtons time to fit:", time.time() - start)
print("Newtons Iterations:", n_model.iterations)
print()

print("Newtons Method Model Summary:\n")
display(n_model.summary())

Newtons time to fit: 0.03992152214050293
Newtons Iterations: 135

Newtons Method Model Summary:



,Variable,Coefficient,Std Error,z-statistic,p-value,[0.025,0.075]
0,Intercept,-0.8608,0.2993,-2.8760,0.004,-1.4474,-0.2742
1,x0,7.9023,0.8969,8.8111,0.000,6.1445,9.6601
2,x1,2.4758,0.4376,5.6576,0.000,1.6181,3.3335


In [11]:
# fit model using lbfgs
# does NOT require tuning in the `turtles` implementation
l_model = LogReg(method="lbfgs")

In [12]:
start = time.time()
l_model.fit(Xtrain, Ytrain)
print("L-BFGS time to fit:", time.time() - start)
print("L-BFGS Iterations:", l_model.iterations)
print()

print("L-BFGS Model Summary:\n")
display(l_model.summary())

L-BFGS time to fit: 0.007733583450317383
L-BFGS Iterations: 13

L-BFGS Model Summary:



,Variable,Coefficient,Std Error,z-statistic,p-value,[0.025,0.075]
0,Intercept,-0.8608,0.2993,-2.8761,0.004,-1.4475,-0.2742
1,x0,7.9024,0.8969,8.8111,0.000,6.1445,9.6602
2,x1,2.4758,0.4376,5.6576,0.000,1.6181,3.3335


In [13]:
# log some other model attributes:
print("Observations:", l_model.observations)
print("Dimensions:", l_model.dimensions)
print("Degrees of Freedom:", l_model.degrees_of_freedom)
print("Critical Z:", l_model.critical_z)

Observations: 455
Dimensions: 2
Degrees of Freedom: 452
Critical Z: 1.959963984540054


In [14]:
# results on test data using 0.5 as threshold
preds = g_model.predict(Xtest) > 0.5
n_preds = n_model.predict(Xtest) > 0.5
l_preds = l_model.predict(Xtest) > 0.5

print("Gradient Descent TEST Accuracy:", np.sum(preds == Ytest) / len(Ytest))
print("Newtons TEST Accuracy:", np.sum(n_preds == Ytest) / len(Ytest))
print("L-BFGS TEST Accuracy:", np.sum(l_preds == Ytest) / len(Ytest))

Gradient Descent TEST Accuracy: 0.9473684210526315
Newtons TEST Accuracy: 0.9473684210526315
L-BFGS TEST Accuracy: 0.9473684210526315


## Compare to Statsmodels

Let's compare the `turtles` implementation to `statsmodels`.

In [15]:
import statsmodels.api as sm

In [16]:
Xtrain = sm.add_constant(Xtrain)

start = time.time()
log_reg = sm.Logit(Ytrain, Xtrain).fit()
print()
print("Statsmodels time to converage: ", time.time() - start)
print()
print(log_reg.summary())

Optimization terminated successfully.
         Current function value: 0.172636
         Iterations 9

Statsmodels time to converage:  0.012334108352661133

                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  455
Model:                          Logit   Df Residuals:                      452
Method:                           MLE   Df Model:                            2
Date:                Mon, 10 Mar 2025   Pseudo R-squ.:                  0.7359
Time:                        17:41:01   Log-Likelihood:                -78.549
converged:                       True   LL-Null:                       -297.42
Covariance Type:            nonrobust   LLR p-value:                 8.823e-96
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.8608      0.299     -2.876      0.00

In [17]:
Xtest = sm.add_constant(Xtest)
sm_preds = log_reg.predict(Xtest) > 0.5
sm_preds = sm_preds.reshape((Ytest.shape[0], 1))

print("Statsmodels TEST Accuracy:", np.sum(sm_preds == Ytest) / len(Ytest))

Statsmodels TEST Accuracy: 0.9473684210526315
